<a href="https://colab.research.google.com/github/D4N18L/Tumor_Detector_CNN_Model/blob/main/BCC_BreakHis_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 pip install tensorflow-gpu==2.4-rc0

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


Import Libraries 

In [ ]:
import keras
from keras.preprocessing import image
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten , BatchNormalization
from keras.utils import np_utils , to_categorical
from sklearn.metrics import accuracy_score

Load Dataset

In [ ]:
loadata = pd.read_csv("/content/gdrive/MyDrive/Folds.csv")
loadata = loadata.sample(frac=1)
loadata.head()
path = "../content/gdrive/MyDrive/BreaKHis_v1/" 

Catgorize and Split Data 

In [ ]:

load_dataset = []
labels = []

for i in tqdm(range(loadata.shape[0])):
    img = image.load_img(path + loadata['filename'].iloc[i], target_size=(28,28,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    load_dataset.append(img)
    
    if (loadata['filename'].iloc[i].find('benign') != -1): 
        labels.append(0) 
    else:
        labels.append(1)
        
X = np.array(load_dataset)
y = np.array(labels)

 
#Splitting the data in train, test  and validation splits
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, random_state=42, test_size=0.2 , shuffle=True)


Y_train = np_utils.to_categorical(y_train, 2)
Y_test = np_utils.to_categorical(y_test, 2)
Y_val = np_utils.to_categorical(y_val, 2)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

100%|██████████| 39545/39545 [2:13:36<00:00,  4.93it/s]


(31636, 28, 28, 3)
(6327, 28, 28, 3)
(1582, 28, 28, 3)


In [ ]:
#Initialization
numepochs = 5
numfilter = 30
kernel = 4
mpool_size = 1

  model = Sequential()
  model.add(Conv2D(numfilter,kernel_size = (3,3),strides=(1,1), padding='valid', activation=ac_function, input_shape=(28,28,3)))
  model.add(MaxPool2D(pool_size=mpool_size))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))


  model.add(Conv2D(numfilter,kernel_size = (3,3),strides=(1,1), padding='valid', activation=ac_function))
  model.add(MaxPool2D(pool_size=mpool_size))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))


  model.add(Conv2D(numfilter,kernel_size = (3,3),strides=(1,1), padding='valid', activation=ac_function))
  model.add(MaxPool2D(pool_size=mpool_size))
  model.add(BatchNormalization())
  model.add(Dropout(0.2))

  model.add(Flatten())
  model.add(Dense(150, activation=ac_function))
  model.add(Dropout(0.2))
  model.add(Dense(2,activation='softmax'))


Generate Summary of the archictectire built

In [ ]:
model.summary()

Compile model

In [ ]:
model.compile(optimizer='adam',loss ='binary_crossentropy',metrics=['accuracy'])
history = model.fit(X_train, Y_train,epochs=numepochs, batch_size=200,validation_data=(X_test,Y_test),verbose=1)

Epoch 1/5
159/159 [==============================] - 189s 1s/step - loss: 0.6360 - accuracy: 0.7767 - val_loss: 0.6283 - val_accuracy: 0.6883
Epoch 2/5
159/159 [==============================] - 185s 1s/step - loss: 0.5649 - accuracy: 0.8031 - val_loss: 0.6205 - val_accuracy: 0.6883
Epoch 3/5
159/159 [==============================] - 188s 1s/step - loss: 0.5233 - accuracy: 0.8123 - val_loss: 0.6265 - val_accuracy: 0.6883
Epoch 4/5
159/159 [==============================] - 184s 1s/step - loss: 0.5017 - accuracy: 0.8097 - val_loss: 0.5389 - val_accuracy: 0.7637
Epoch 5/5
159/159 [==============================] - 184s 1s/step - loss: 0.4883 - accuracy: 0.8085 - val_loss: 0.5152 - val_accuracy: 0.7700


# **Initialize Model Name **

In [ ]:
Modelname = "BCC_BreakHis.h5"

# **Save Model**

In [ ]:
model.save(Modelname)

# **Test Model**

In [ ]:
Test_loss,Test_accuaracy = model.evaluate(X_test,Y_test,verbose=1)
print("" , Test_accuaracy)

198/198 [==============================] - 7s 37ms/step - loss: 0.1539 - accuracy: 0.9551
 0.9551129937171936


# **Convert TensorFlow Model into TFLITE **

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:1397: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: /tmp/tmprtl3x3w3/assets


# **SAVE TFLITE MODEL**

In [ ]:
with open('CNN_BreakHis.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
labels = ['benign', 'malignant']

with open('labels.txt', w) as f:
  f.write('\n'.join(labels))